## P0 Symbol Table
#### Original Author: Emil Sekerinski, revised March 2024
Declarations of the source program are entered into the symbol table as the source program is parsed. The symbol detects multiple definitions or missing definitions and reports those by calling procedure `mark(msg)` of the scanner.
- classes `Var`, `Ref`, `Const`, `Type`, `Proc`, `StdProc` are for the symbol table entires
- classes `Int`, `Bool`, `Record`, `Array` are for the types of symbol table entries
- procedures `Init()`, `newDecl(name, entry)`, `find(name)`, `openScope()`, `topScope()`, `closeScope()` are the operations of the symbol table
- procedure `printSymTab()` visualizes the symbol table in a readable textual form with indentation.

In [ ]:
import nbimporter, textwrap
nbimporter.options["only_defs"] = False
from SC import mark

def indent(n):
    return textwrap.indent(str(n), '  ')

Symbol table entries are objects of following classes:
- `Var` for global variables, local variables, and value parameters (must be `Int` or `Bool`)
- `Ref` for reference parameters (of any type)
- `Const` for constants of types `Int` or `Bool`
- `Type` for named or anonymous types
- `Proc` for declared procedures
- `StdProc` for one of `write`, `writeln`, `read`

All entries have a field `tp` for the type, which can be `None`.

In [ ]:
class Var:
    def __init__(self, tp):
        self.tp = tp
        self.belongsToClass = False
        self.cl = Class("")
        
    def setClass(self, c):
        self.belongsToClass = True
        self.cl = c
        
    def __str__(self):
        return 'Var(name = ' + str(getattr(self, 'name', '')) + ', lev = ' + \
                str(getattr(self, 'lev', '')) + ', tp = ' + str(self.tp) + \
                ', part of class ' + str(self.belongsToClass) + ', from Class: ' + \
                str(self.cl.name) + ', adr: '+ str(getattr(self, 'adr', '')) + \
                ', reg: '+ str(getattr(self, 'reg', '')) + ')'
    def __eq__(self, other):
        return self.reg == other.reg and self.adr == other.adr
    def __hash__(self):
        return hash(str(self))


#not used, might be of use
class Object(Var):
    def __init__(self, tp):
        super().__init__(tp)
    def __str__(self):
        return 'Object(name = ' + str(getattr(self, 'name', '')) + ', lev = ' + \
               str(getattr(self, 'lev', '')) + ', tp = ' + str(self.tp) + ')'

class Ref:
    def __init__(self, tp):
        self.tp = tp
    def __str__(self):
        return 'Ref(name = ' + str(getattr(self, 'name', '')) + ', lev = ' + \
               str(getattr(self, 'lev', '')) + ', tp = ' + str(self.tp) + \
                ', adr: '+ str(getattr(self, 'adr', '')) + \
                ', reg: '+ str(getattr(self, 'reg', '')) + ')'

class Res:
    def __init__(self, tp):
        self.tp = tp
    def __str__(self):
        return 'Res(name = ' + str(getattr(self, 'name', '')) + ', lev = ' + \
               str(getattr(self, 'lev', '')) + ', tp = ' + str(self.tp) + ')'

class Const:
    def __init__(self, tp, val):
        self.tp, self.val = tp, val
    def __str__(self):
        return 'Const(name = ' + str(getattr(self, 'name', '')) + ', tp = ' + \
               str(self.tp) + ', val = ' + str(self.val) + ')'
    def __eq__(self, other):
        return self.val == other.val
    def __hash__(self):
        return hash(str(self))

class Type:
    def __init__(self, tp):
        self.tp, self.val = None, tp
    def __str__(self):
        return 'Type(name = ' + str(getattr(self, 'name', '')) + ', val = ' + \
               str(self.val) + ')'

class Proc:
    def __init__(self, par, res, classMet = False):
        self.tp, self.par, self.res = None, par, res
        self.classMet = classMet
    def __str__(self):
        return 'Proc(name = ' + self.name + ', lev = ' + str(self.lev) + \
               ', par = [' + ', '.join(str(s) for s in self.par) + ']' + \
               ', res = [' + ', '.join(str(s) for s in self.res) + ']' + \
               ', class method? ' + str(self.classMet) + ')'

class StdProc:
    def __init__(self, par, res):
        self.tp, self.par, self.res = None, par, res
    def __str__(self):
        return 'StdProc(name = ' + self.name + ', lev = ' + str(self.lev) + \
               ', par = [' + ', '.join(str(s) for s in self.par) + ']' + \
               ', res = [' + ', '.join(str(s) for s in self.res) + '])'
    
class Class(Type):
    def __init__(self, name, extends = ""):
        super().__init__(self)
        self.name = name
        self.extends = extends
        self.att= []
        self.methods = []
        self.size = 0
    def __str__(self):
        return 'Class(name = ' + self.name + ', extends = ' + self.extends + \
                ', lev = ' + str(self.lev) + \
                ', att = [' + ', '.join(str(s) for s in self.att) + ']' + \
                ', methods = [' + ', '.join(str(s) for s in self.methods) + ']' + \
                ', size = ' + str(self.size) + ')'
    

- the P0 types `integer` and `boolean` are represented by the classes `Int` and `Bool`; no objects of `Int` or `Bool` are created
- record and array types in P0 are represented by objects of class `Record` and `Array`; for records, a list of fields is kept, for arrays, the base type, the lower bound, and the length of the array is kept.

In [ ]:
class Int: pass

class Bool: pass

class Enum: pass # for adding enumeration types

class Record:
    def __init__(self, fields):
        self.fields = fields
    def __str__(self):
        return 'Record(fields = [' + ', '.join(str(f) for f in self.fields) + '])'

class Array:
    def __init__(self, base, lower, length):
        self.base, self.lower, self.length = base, lower, length
    def __str__(self):
        return 'Array(lower = ' + str(self.lower) + ', length = ' + \
               str(self.length) + ', base = ' + str(self.base) + ')'

The symbol table is represented by a list of scopes. Each scope is a list of entries. Each entry has a name, which is assumed to be a string, and the level at which it is declared; the entries on the outermost scope are on level 0 and the level increases with each inner scope.

In [ ]:
def init():
    global symTab, classDict
    symTab = [[]]
    classDict = {}

def printSymTab():
    print('Symbol table::', type(symTab))
    for l in symTab:
        for e in l: print(' :', e)
        print()

def newDecl(name, entry):
    top, entry.lev, entry.name = symTab[0], len(symTab) - 1, name
    for e in top:
        if e.name == name:
            mark("multiple definition of " + str(name)); return
    top.append(entry)

def find(name):
    for l in symTab:
        for e in l:
            if name == e.name: return e
    if '.' not in name:
        mark('undefined identifier ' + name)
    else:
        # . in name means related to classes, class.attribute or class.procedure
        try:
            return findInClass(name)
        except:
            #print("did not find in attributes")
            return findMethod(name)
    return Const(None, 0)

#return a Proc type
def findMethod(name):
    # e.g.   j.setPrintSIN
    nameSplit = name.split('.')
    if len(nameSplit) != 2: mark('undeinfed class identifier' + name)
    varName = nameSplit[0] #e.g. j
    metName = nameSplit[1] #e.g. setPrinSIN
    for l in symTab:
        found = False
        for e in l:
            if e.name == varName:
                found = True
                cl = e.tp
                assert(type(cl) == Class)
                break
        if found: break
    if not found: mark('undefined object identifier' + varName)
    while True:
        # class.methodName
        procName = cl.name + '.' + metName
        for l in symTab:
            for e in l:
                if procName == e.name: 
                    assert(type(e) == Proc)
                    return e
        #if not found, might be base class method
        found = False
        for l in symTab:
            for e in l:
                if cl.extends == e.name:
                    assert(type(e) == Class)
                    cl = e
                    found = True
                    break
            if found: break
        if not found:
            mark("Method not found " + name)

#return a Var type
def findInClass(name):
    #'student.grade'
    nameSplit = name.split('.')
    if len(nameSplit) != 2: mark('undeinfed class identifier' + name)
    varName = nameSplit[0]
    class_att_met = nameSplit[1]
    for l in symTab:
        for e in l:
            #found the Var type
            if varName == e.name:
                objAdr = e.adr
                objLev = e.lev
                objTp = e.tp
                objReg = e.reg
                for attrib in objTp.att:
                    #found the tuple (name, type, offset)
                    if attrib[0] == class_att_met:
                        tp = attrib[1]
                        offset = attrib[2]
                        vari = Var(tp)
                        vari.lev = objLev
                        vari.name = name
                        if type(objAdr) == str:
                            try:
                                assert '_' in objAdr
                            except:
                                print('Str type objAdr, but no _:', objAdr)
                                mark('Str type objAdr, but no _:' + objAdr)
                            vari.adr = objAdr + '+' + str(offset)
                        else:
                            try:
                                assert type(objAdr) == int
                            except:
                                print('Str type objAdr, but no _:', type(objAdr))
                                mark('objAdr not int type' + str(objAdr))
                            vari.adr = objAdr + offset
                        vari.reg = objReg
                        return vari
    mark("undefined identifier" + name)

def openScope():
    symTab.insert(0, [])

def topScope():
    return symTab[0]

def closeScope():
    symTab.pop(0)
    
def newClass(name):
    pass